In [2]:
library(dplyr)
library(data.table)
library(ggplot2)
library(tidyverse)
library(lubridate)
library(patchwork)
library(clusterAnalysisUtils)
library(MASS)
library(robmixglm)
library(mvabund)
library(fitdistrplus)
library(lattice)
library(lmtest)
library(pscl)
library(emmeans)
library(multcompView)
library(multcomp)
library(ggpubr)
library(psycho)
library(ggnewscale)


format_p <- function(s) {
 out <- symnum(s, 
                 symbols   = c("***","**","*",".","ns"),
                 cutpoints = c(0,  .001,.01,.05, .1, 1),
                 corr      = FALSE
               )
 return(out)
 }

In [3]:
#Collect meta data
data <- lapply(X = snakemake@input$datas, FUN = function(data){
    df = fread(data) %>%
        dplyr::select(taxon,country,date) %>%
        as.data.table()
    return(df)
})
data <- bind_rows(data)

In [4]:
#Convert all dates to days from the earliest
data$date <- as.Date(data$date)
min_date = min(data$date)
data$days = as.numeric(data$date - min_date)
data$weeks = round((data$days/7),digits = 0)

In [5]:
#reda clusters
dfs <- lapply(X = snakemake@input$ids, FUN = function(data){
    sm <- strsplit(x=data, split="/")[[1]][[4]]
    df = clusterAnalysisUtils::clusterDataParser$new(data,format = "UC",remove_sizes = T)$df
    df$Sample <- sm
    return(df)
})
df <- bind_rows(dfs)

In [6]:
df <- df %>%
    left_join(data,by=c("Member"="taxon")) %>%
    as.data.table()
    

In [7]:
df <- df %>%
    group_by(Sample,Cluster) %>%
    summarise(N=n(),D=max(days)-min(days)) %>%
    ungroup() %>%
    as.data.table()

head(df)

In [8]:
countdf <- df %>%
    group_by(Sample) %>%
    summarise(N=n(),D=mean(D,na.rm = T)) %>%
    ungroup() %>%
    as.data.table()
countdf

In [9]:
#general data on cluster with more than 1 member
dfwo0 <- filter(df,N>1) %>%
    as.data.table()

data = dfwo0
test = data %>%
    group_by(Sample) %>%
    summarise(Number=n(),
              DD=mean(D,na.rm = T),
              `Maximum of N`=max(N),
              `Minimum of N`=min(N),
              `Mean of N`=mean(N),
              `Median of N`=median(N),
              `Maximum of D`=max(D,na.rm = T),
              `Minimum of D`=min(D,na.rm = T),
              `Mean of D`=mean(D,na.rm = T),
              `Median of D`=median(D,na.rm = T),

              ) %>%
    mutate(
        `Mean of N`=round(`Mean of N`,digits = 2),
    ) %>%
    as.data.table()
test

In [10]:
options(repr.plot.width = 10, repr.plot.height = 4, jupyter.plot_mimetypes = "image/svg+xml")
#get initial situation with densities - HISTOGRAM
data=dfwo0
pd <- ggplot(data,aes(x=D,color=Sample)) + geom_density()
plotd <- as.data.table(ggplot_build(pd)$data) %>%
    group_by(colour) %>%
    summarise(x=max(x[which.max(y)])) %>%
    as.data.table() 
    
pd <- pd + geom_vline(xintercept=plotd$x,color=plotd$colour) + xlab("D, days")
pb <- ggplot(data, aes(x=Sample, y=D)) +
    geom_boxplot(aes(fill=Sample)) +
    guides(fill="none") +
    ylab("D, days")
print(plotd)
(pd | pb) + plot_annotation(title = 'Distribution identical sequence existance (D)')


In [11]:
options(repr.plot.width = 10, repr.plot.height = 4, jupyter.plot_mimetypes = "image/svg+xml")
data=dfwo0

pd <- ggplot(data,aes(x=N,color=Sample)) + geom_density(adjust=2) + xlim(0,15)

plotd <- as.data.table(ggplot_build(pd)$data) %>%
    group_by(colour) %>%
    summarise(x=max(x[which.max(y)])) %>%
    as.data.table()
pd <- pd + geom_vline(xintercept=plotd$x,color=plotd$colour) + xlab("N, number") 
pb <- ggplot(data, aes(x=Sample, y=N)) +
    geom_boxplot(aes(fill=Sample)) +
    guides(fill="none") + scale_y_log10() +
    ylab("N, number")
print(plotd)
(pd | pb) + plot_annotation(title = 'Distribution of cluster sizes (N)')


In [12]:
# get limit of Ns
data= dfwo0
test <- data %>%
    group_by(Sample) %>%
    summarise(PC = list(quantile(N))) %>% 
    as.data.table() %>%
    rowwise() %>%
    mutate(minl=PC[[2]], maxl=PC[[4]])
minl = max(test$minl)
maxl = min(test$maxl)
dfwoe = dfwo0 %>%
    filter(N >=  minl & N <= maxl) %>%
    as.data.table()


In [13]:
#Data  on the filteredf data

data = dfwoe
test = data %>%
    group_by(Sample) %>%
    summarise(Number=n(),
              `Maximum of N`=max(N),
              `Minimum of N`=min(N),
              `Mean of N`=mean(N),
              `Median of N`=median(N),
              `Maximum of D`=max(D,na.rm = T),
              `Minimum of D`=min(D,na.rm = T),
              `Mean of D`=mean(D,na.rm = T),
              `Median of D`=median(D,na.rm = T),

              ) %>%
    mutate(
        `Mean of N`=round(`Mean of N`,digits = 2),
    ) %>%
    as.data.table()
test

In [14]:
options(repr.plot.width = 10, repr.plot.height = 4, jupyter.plot_mimetypes = "image/svg+xml")
data=dfwoe

pd <- ggplot(data,aes(x=N,fill=Sample)) + geom_histogram(binwidth = 1) +
    facet_grid(rows = vars(Sample),scales = "free") + xlab("N, number")


pb <- ggplot(data, aes(x=Sample, y=N)) +
    geom_boxplot(aes(fill=Sample)) +
    guides(fill="none") +
    ylab("N, number")
(pd | pb) + plot_annotation(title = 'Distribution of cluster sizes (N)  after cluster size match')

In [15]:
options(repr.plot.width = 10, repr.plot.height = 4, jupyter.plot_mimetypes = "image/svg+xml")
#get initial situation with densities - HISTOGRAM
data=dfwoe
pd <- ggplot(data,aes(x=D,color=Sample)) + geom_density()
plotd <- as.data.table(ggplot_build(pd)$data) %>%
    group_by(colour) %>%
    summarise(x=max(x[which.max(y)])) %>%
    as.data.table() 
    
pd <- pd + geom_vline(xintercept=plotd$x,color=plotd$colour) + xlab("D, days")
pb <- ggplot(data, aes(x=Sample, y=D)) +
    geom_boxplot(aes(fill=Sample)) +
    guides(fill="none") +
    ylab("D, days")
print(plotd)
(pd | pb) + plot_annotation(title = 'Distribution identical sequence existance (D) after cluster size match')

In [16]:
outondist = list()

In [17]:
## Poisson GLM
M1 <- glm(D ~ Sample,
          family = 'poisson',
          data = data)

M1sum<-summary(M1)
P <- M1sum$coefficients[[2,4]]
P
## Check for over/underdispersion in the model
E2 <- resid(M1, type = "pearson")
N  <- nrow(data)
p  <- length(coef(M1))   
Res <- sum(E2^2) / (N - p)
#add_row(outondist, Distribution="Poisson GLM", P=p,`Residual mean deviance`=Res )
Res
outondist[[1]]=list("Poison GLM",P,Res)

In [18]:
## Negative Binomial GLM
M2 <- glm.nb(D ~ Sample,
             data = data)
M2sum <- summary(M2)
P <- M2sum$coefficients[[2,4]]
# Dispersion statistic
E2 <- resid(M2, type = "pearson")
N  <- nrow(data)
p  <- length(coef(M2)) + 1  # '+1' is for variance parameter in NB
Res <- sum(E2^2) / (N - p)
outondist[[2]]=list("Negative Binomial GLM",P,Res)


In [19]:
## Zero-Inflated Poisson GLM
M3 <- zeroinfl(D ~ Sample | ## Predictor for the Poisson process
                 Sample, ## Predictor for the Bernoulli process;
               dist = 'poisson',
               data = data)

M3sum <- summary(M3)
P <- M3sum$coefficients$count[[2,4]]
P
M3sum
# Dispersion statistic
E3 <- resid(M3, type = "pearson")
N  <- nrow(data)
p  <- length(coef(M3))  
Res <- sum(E2^2) / (N - p)
outondist[[3]]=list("Zero-Inflated Poisson GLM",P,Res)

In [20]:
M4 <- zeroinfl(D ~ Sample |
                 Sample,
               dist = 'negbin',
               data = data)
M4sum <- summary(M4)
P <- M4sum$coefficients$count[[2,4]]
M4sum
# Dispersion Statistic
E2 <- resid(M4, type = "pearson")
N  <- nrow(data)
p  <- length(coef(M4)) + 1 # '+1' is due to theta

Res <- sum(E2^2) / (N - p)
outondist[[4]]=list("Zero-Inflated Negative Binomial GLM",P,Res)

In [21]:
outondist4show <- as.data.frame(do.call(rbind, outondist))
names(outondist4show) <- c("Distribution","P","Residual mean deviance")
outondist4show

In [22]:
# get a mock data for playing 
n=1000
d1t=rpois(n, 2) 
da = data.table(D=d1t,Sample="A")
d2t=rpois(n/10,1)
db = data.table(D=d2t,Sample="B")
d3t=rpois(n/5,1)
dc = data.table(D=d3t,Sample="C")
d4t=rpois(n,1.5)
dd = data.table(D=d3t,Sample="D")

dftest = bind_rows(da,db,dc,dd) %>%
    as.data.table()

In [68]:
# run Wilcoxon anova and format output
data = as.data.frame(dfwoe)
ref = "BA.2"
p_limit = 0.10

data_sum <- data %>%
    group_by(Sample) %>%
    summarise(Median = median(D), Mean = mean(D))
data_sum
#test wilcoxon
wilcox_less <- compare_means(D~Sample, data=data, alternative="less",ref.group = ref) %>%
    filter(p.adj <= p_limit)
wilcox_less$Hypothesis <- paste( "Less than",ref,sep=" ")

wilcox_greater <- compare_means(D~Sample, data=data, alternative="greater",ref.group = ref) %>%
    filter(p.adj <= p_limit)
wilcox_greater$Hypothesis <- paste( "Greater than",ref,sep=" ")

wilcox_equal <- compare_means(D~Sample, data=data) %>%
    filter(p.adj <= p_limit)
wilcox_equal$Hypothesis <- paste( "Not equal")
wilcox_tests <- bind_rows(wilcox_less, wilcox_greater,wilcox_equal)
wilcox_tests <- wilcox_tests %>%
    filter(p.adj <= p_limit)
wilcox_tests

In [91]:
kt = kruskal.test(D~Sample, data = data)


In [104]:


data_sum <- data_sum %>%
    arrange(Mean)
data_sum
data$Sample <- factor(data$Sample, levels = data_sum$Sample)
pleg <- attr(format_p(0.1),"legend")


ggboxplot(data, x = "Sample", y = "D",
          color = "Sample", palette = "jco", alpha=0.0 )+ 
        #ylim(0,quantile(data$D,0.95)) +
    scale_color_discrete(guide = 'none') +
    new_scale_color() + 
    stat_pvalue_manual(
        wilcox_tests, 
        y.position = quantile(data$D,0.95), step.increase = 0.1,
        label = "p.signif",
        color="Hypothesis",
        hide.ns = F,
    )  +
    ylab("Days") +
    xlab("Lineage") +
    ggtitle(paste("Kruskal-Wallis test,  p-value",format.pval(kt$p.value))) +
    theme(plot.title = element_text(size = 12, face = "bold")) +
    labs(caption = paste("p-value legend:",leg)) +
    stat_summary(fun.data = function(x) data.frame(y=32, label = paste(round(mean(x), digits = 2))),
                                                   geom="text",alpha = 0.5)





In [29]:
# run ZINB anova and format output
data = as.data.frame(dfwoe)

ref = "BA.2"

p_limit = 0.10


data_sum <- data %>%
    group_by(Sample) %>%
    summarise(Median = median(D), Mean = mean(D))

data_sum


zng <- zeroinfl(D ~ Sample |
                 Sample,
               dist = 'negbin',
               data = data)

marginal = emmeans(zng,
                   ~ Sample)

mod.pairs = pairs(marginal,
      adjust="tukey")

mod.cld =  multcomp::cld(marginal,
    alpha=0.05,
    Letters=letters,  ### Use lower-case letters for .group
    adjust="tukey")   ### Tukey adjustment for multiple comparison

df_zinb_contrast <- as.data.frame(mod.pairs) %>%
    separate(col = contrast,sep="-", into = c("group1","group2")) %>%
    rename(p = p.value) %>%
    mutate(p.adj = p) %>%
    mutate(p.format = format.pval(p)) %>%
    mutate(group1 = str_replace_all(group1," ","")) %>%
    mutate(group2 = str_replace_all(group2," ","")) %>%
    mutate(method="ZINB", Hypothesis = "Not equal", .y. = 'D') %>%
    mutate(p.signif = format_p(p))

df_zinb_contrast

df_zinb_emean <- as.data.frame(mod.cld)

df_zinb_emean$Sample <- factor(df_zinb_emean$Sample, levels = df_zinb_emean$Sample)

df_zinb_emean

In [98]:
sig_l = 0.1
df_zinb_contrast4p <- df_zinb_contrast %>%
    filter(p<=sig_l)
pvall <-  summary(zng)$coefficients$count[[2,4]]
leg <- print(attr(df_zinb_contrast4p$p.signif,"legend"))


p <- ggplot(data = df_zinb_emean)+ 
    geom_errorbar(aes(ymin = asymp.LCL, ymax = asymp.UCL, x=Sample, color=Sample), width = 0.3) + 
    geom_point(aes(x=Sample, y= emmean)) +
    ylab("Estimated Marginal Mean, days") +
    scale_color_discrete(guide='none') +
    new_scale_color() +
    scale_color_discrete(name='Pairwise p-value') +
    stat_pvalue_manual(
        df_zinb_contrast4p, 
        y.position = max(df_zinb_emean$asymp.UCL), step.increase = 0.1,
        label = "p.signif",
        color="p.signif",
        hide.ns = F,
    ) +
    xlab("Lineage") +
    ggtitle(paste("Zero-Inflated Negative Binomial GLM,  p-value",format.pval(pvall))) +
    theme(plot.title = element_text(size = 12, face = "bold")) 


p <- p  +  labs(caption = paste("p-value legend:",leg))
p